In [ ]:
# ## Prerequisite. Run on Terminal. Install miniconda. Navigate to the folder cd /home/asus/

# mkdir -p ~/miniconda3
# wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O ~/miniconda3/miniconda.sh
# bash ~/miniconda3/miniconda.sh -b -u -p ~/miniconda3
# rm -rf ~/miniconda3/miniconda.sh
# ~/miniconda3/bin/conda init bash
# ~/miniconda3/bin/conda init zsh

In [ ]:
#### RUn on terminal: Create a virtual environment, Install all the requirements like pytorch, xformers, cuda etc.
#### install the unsloth library and other libraries viz. peft

# conda create --name unsloth_env python=3.11 pytorch-cuda=12.1 pytorch cudatoolkit xformers -c pytorch -c nvidia -c xformers -y
# conda activate unsloth_env
# pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
# pip install --no-deps trl peft accelerate bitsandbytes

In [1]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/DeepSeek-R1-Distill-Llama-8B-unsloth-bnb-4bit",
    "unsloth/DeepSeek-R1-Distill-Qwen-14B-unsloth-bnb-4bit",
    "unsloth/DeepSeek-R1-Distill-Qwen-7B-unsloth-bnb-4bit",
    "unsloth/phi-4-unsloth-bnb-4bit",
    "unsloth/Mistral-Small-24B-Instruct-2501-unsloth-bnb-4bit",
    "unsloth/Meta-Llama-3.1-8B-Instruct-unsloth-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-unsloth-bnb-4bit",
    "unsloth/Llama-3.2-1B-Instruct-unsloth-bnb-4bit",
    "unsloth/Mistral-Small-24B-Base-2501-unsloth-bnb-4bit",
    "unsloth/Llama-3.1-8B-Instruct-unsloth-bnb-4bit",
    "unsloth/Llama-3.1-8B-unsloth-bnb-4bit",
    "unsloth/Llama-3.2-11B-Vision-Instruct-unsloth-bnb-4bit",
    "unsloth/Qwen2.5-VL-7B-Instruct-unsloth-bnb-4bit",
    "unsloth/Qwen2.5-VL-72B-Instruct-unsloth-bnb-4bit",
    "unsloth/Qwen2.5-VL-3B-Instruct-unsloth-bnb-4bit",
    "unsloth/Qwen2-VL-7B-Instruct-unsloth-bnb-4bit",
    "unsloth/Qwen2-VL-2B-Instruct-unsloth-bnb-4bit",
    "unsloth/Pixtral-12B-2409-unsloth-bnb-4bit",
    "unsloth/Llama-3.2-11B-Vision-unsloth-bnb-4bit",
    "unsloth/QwQ-32B-Preview-unsloth-bnb-4bit",
    "unsloth/Qwen2.5-3B-Instruct-unsloth-bnb-4bit",
] # More models at https://huggingface.co/unsloth

In [2]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/DeepSeek-R1-Distill-Qwen-7B-unsloth-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2025.2.15: Fast Qwen2 patching. Transformers: 4.49.0.
   \\   /|    GPU: NVIDIA GeForce RTX 4070. Max memory: 11.994 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1. CUDA: 8.9. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.2.15 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [4]:
alpaca_prompt= """Below is an instruction that give an sql prompt. Write a response that appropriately completes the request and gives you an sql and the corresponding explanation.

### sql_prompt:
{}

### sql:
{}

### Explanation:
{}"""

In [5]:
alpaca_prompt

'Below is an instruction that give an sql prompt. Write a response that appropriately completes the request and gives you an sql and the corresponding explanation.\n\n### sql_prompt:\n{}\n\n### sql:\n{}\n\n### Explanation:\n{}'

In [6]:
def formatting_prompts_func(examples):
    sql_prompt = examples["sql_prompt"]
    sql = examples["sql"]
    sql_explanation = examples['sql_explanation']

    texts = []
    
    EOS_TOKEN = tokenizer.eos_token
    for sql_prompt, sql, sql_explanation in zip(sql_prompt, sql, sql_explanation):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(sql_prompt, sql, sql_explanation) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

In [7]:
from datasets import load_dataset
dataset = load_dataset("gretelai/synthetic_text_to_sql", split = "train")
print(dataset.column_names)

['id', 'domain', 'domain_description', 'sql_complexity', 'sql_complexity_description', 'sql_task_type', 'sql_task_type_description', 'sql_prompt', 'sql_context', 'sql', 'sql_explanation']


In [8]:
dataset

Dataset({
    features: ['id', 'domain', 'domain_description', 'sql_complexity', 'sql_complexity_description', 'sql_task_type', 'sql_task_type_description', 'sql_prompt', 'sql_context', 'sql', 'sql_explanation'],
    num_rows: 100000
})

In [9]:
dataset = dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [10]:
dataset

Dataset({
    features: ['id', 'domain', 'domain_description', 'sql_complexity', 'sql_complexity_description', 'sql_task_type', 'sql_task_type_description', 'sql_prompt', 'sql_context', 'sql', 'sql_explanation', 'text'],
    num_rows: 100000
})

In [11]:
dataset["text"][0:4]

['Below is an instruction that give an sql prompt. Write a response that appropriately completes the request and gives you an sql and the corresponding explanation.\n\n### sql_prompt:\nWhat is the total volume of timber sold by each salesperson, sorted by salesperson?\n\n### sql:\nSELECT salesperson_id, name, SUM(volume) as total_volume FROM timber_sales JOIN salesperson ON timber_sales.salesperson_id = salesperson.salesperson_id GROUP BY salesperson_id, name ORDER BY total_volume DESC;\n\n### Explanation:\nJoins timber_sales and salesperson tables, groups sales by salesperson, calculates total volume sold by each salesperson, and orders the results by total volume in descending order.<｜end▁of▁sentence｜>',
 'Below is an instruction that give an sql prompt. Write a response that appropriately completes the request and gives you an sql and the corresponding explanation.\n\n### sql_prompt:\nList all the unique equipment types and their corresponding total maintenance frequency from the eq

In [12]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 2,
        warmup_steps = 3,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps =  60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Converting train dataset to ChatML (num_proc=2):   0%|          | 0/100000 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=2):   0%|          | 0/100000 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/100000 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/100000 [00:00<?, ? examples/s]

In [13]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 4070. Max memory = 11.994 GB.
8.143 GB of memory reserved.


In [14]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 100,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 2
\        /    Total batch size = 2 | Total steps = 60
 "-____-"     Number of trainable parameters = 40,370,176


Step,Training Loss
1,3.151200
2,3.050800
3,3.273700
4,2.721100
5,2.754600
6,2.602500
7,2.528800
8,1.678500
9,1.780100
10,1.683100


In [15]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

68.7814 seconds used for training.
1.15 minutes used for training.
Peak reserved memory = 8.764 GB.
Peak reserved memory for training = 0.621 GB.
Peak reserved memory % of max memory = 73.07 %.
Peak reserved memory for training % of max memory = 5.178 %.


In [16]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

In [17]:
import torch
torch.cuda.empty_cache()


In [ ]:
# model.save_pretrained_gguf("model", tokenizer, quantization_method = "f32")

Follow this Steps for Local Saving

In [19]:
model.save_pretrained_merged("merged_model", tokenizer, save_method = "merged_16bit",)

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 28.16 out of 49.39 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


  4%|█████▊                                                                                                                                                           | 1/28 [00:00<00:09,  2.71it/s]
We will save to Disk and not RAM now.
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 28/28 [00:17<00:00,  1.62it/s]


Unsloth: Saving tokenizer... Done.
Done.


In [20]:
!git clone --recursive https://github.com/ggerganov/llama.cpp

fatal: destination path 'llama.cpp' already exists and is not an empty directory.


In [21]:
!make clean -C llama.cpp

make: Entering directory '/home/ryan/Documents/workspaces/workspace_ai/FineTune/llama.cpp'
Makefile:2: *** The Makefile build is deprecated. Use the CMake build instead. For more details, see https://github.com/ggml-org/llama.cpp/blob/master/docs/build.md.  Stop.
make: Leaving directory '/home/ryan/Documents/workspaces/workspace_ai/FineTune/llama.cpp'


In [22]:
!make all -j -C llama.cpp

make: Entering directory '/home/ryan/Documents/workspaces/workspace_ai/FineTune/llama.cpp'
Makefile:2: *** The Makefile build is deprecated. Use the CMake build instead. For more details, see https://github.com/ggml-org/llama.cpp/blob/master/docs/build.md.  Stop.
make: Leaving directory '/home/ryan/Documents/workspaces/workspace_ai/FineTune/llama.cpp'


In [23]:
!pip install gguf protobuf

In [24]:
!python llama.cpp/convert_hf_to_gguf.py merged_model --outfile text2sql_8.gguf --outtype q8_0

INFO:hf-to-gguf:Loading model: merged_model
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:hf-to-gguf:gguf: loading model part 'model-00001-of-00004.safetensors'
INFO:hf-to-gguf:token_embd.weight,         torch.bfloat16 --> Q8_0, shape = {3584, 152064}
INFO:hf-to-gguf:blk.0.attn_norm.weight,    torch.bfloat16 --> F32, shape = {3584}
INFO:hf-to-gguf:blk.0.ffn_down.weight,     torch.bfloat16 --> Q8_0, shape = {18944, 3584}
INFO:hf-to-gguf:blk.0.ffn_gate.weight,     torch.bfloat16 --> Q8_0, shape = {3584, 18944}
INFO:hf-to-gguf:blk.0.ffn_up.weight,       torch.bfloat16 --> Q8_0, shape = {3584, 18944}
INFO:hf-to-gguf:blk.0.ffn_norm.weight,     torch.bfloat16 --> F32, shape = {3584}
INFO:hf-to-gguf:blk.0.attn_k.bias,         torch.bfloat16 --> F32, shape = {512}
INFO:hf-to-gguf:blk.0.attn_k.weight,       torch.bfloat16 --> Q8_0, shape = {3584, 512}

INSTALL OLLAMA using Terminal

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
[sudo] password for ryan: 

In [25]:
!ollama create unsloth_model_8 -f Modelfile

gathering model components ⠋ gathering model components ⠹ gathering model components ⠹ gathering model components ⠸ gathering model components ⠼ gathering model components ⠴ gathering model components ⠦ gathering model components ⠧ gathering model components ⠇ gathering model components ⠏ gathering model components ⠋ gathering model components ⠙ gathering model components ⠹ gathering model components ⠸ gathering model components ⠼ gathering model components ⠴ gathering model components ⠦ gathering model components ⠧ gathering model components ⠇ gathering model components ⠏ gathering model components ⠙ gathering model components ⠙ gathering model components ⠹ gathering model components ⠸ gathering model components ⠼ gathering model components ⠴ gathering model components ⠧ gathering model components ⠧ gathering model components ⠇ gathering model components ⠋ gathering model components ⠋ gathering model components ⠙ gathering model components ⠹ gathering model components ⠸ gathering mode

In [26]:
alpaca_prompt1= """Below is an instruction that give an sql prompt. Write a response that appropriately completes the request and gives you an sql and the corresponding explanation.

### sql_prompt:
{}

### sql:
{}

### Explanation:
{}"""

In [27]:
input = alpaca_prompt1.format(
    "What is the total volume of timber sold by each salesperson, sorted by salesperson?",
    "",
    "",)

In [28]:
input

'Below is an instruction that give an sql prompt. Write a response that appropriately completes the request and gives you an sql and the corresponding explanation.\n\n### sql_prompt:\nWhat is the total volume of timber sold by each salesperson, sorted by salesperson?\n\n### sql:\n\n\n### Explanation:\n'

In [29]:
query = "What is the average property size in inclusive housing areas?"

In [30]:
input = alpaca_prompt1.format(
    query,
    "",
    "",)

In [31]:
input

'Below is an instruction that give an sql prompt. Write a response that appropriately completes the request and gives you an sql and the corresponding explanation.\n\n### sql_prompt:\nWhat is the average property size in inclusive housing areas?\n\n### sql:\n\n\n### Explanation:\n'